In [1]:
import pandas as pd
import numpy as np
from itertools import product

# === Load and normalize matrices ===
matrices = {
    'minutes': pd.read_csv('minutes matrix.csv', index_col=0),
    'win': pd.read_csv('win percentage matrix.csv', index_col=0),
    'passes': pd.read_csv('pass matrix.csv', index_col=0),
    'xg_for': pd.read_csv('xgfor matrix.csv', index_col=0),
    'xg_against': pd.read_csv('xgagainst matrix.csv', index_col=0),
    'shot_assist': pd.read_csv('shotassist matrix.csv', index_col=0),
    'shot_dribble': pd.read_csv('shots_dribbles_per90_matrix.csv', index_col=0),
    'carries': pd.read_csv('carries_per90_matrix.csv', index_col=0),
    'defense': pd.read_csv('defensive_actions_per90_matrix.csv', index_col=0),
    'recoveries': pd.read_csv('ball_recoveries_per90_matrix.csv', index_col=0),
    'pressures': pd.read_csv('pressures_per90_matrix.csv', index_col=0),
}

def normalize_df(df):
    min_val = df.min().min()
    max_val = df.max().max()
    if max_val - min_val == 0:
        return df - min_val
    return (df - min_val) / (max_val - min_val)

for key in matrices:
    matrices[key] = normalize_df(matrices[key])

position_roles = {
    'GK': ['Conservative', 'Shot Stopper', 'Sweeper Keeper'],
    'LB': ['Balanced Fullback', 'Attacking Fullback', 'Defensive Fullback'],
    'RB': ['Balanced Fullback', 'Attacking Fullback', 'Defending Fullback'],
    'CB': ['Stopper', 'Sweeper', 'Enforcer', 'Libero', 'Ball Playing CB'],
    'CDM': ['Ball Winning Midfielder', 'Positional Midfielder', 'Anchor Man', 'Deep Playmaker'],
    'CM': ['Box to Box Midfielder', 'Holding Midfielder', 'Playmaker'],
    'CAM': ['Advanced Playmaker', 'False 10', 'Second Striker'],
    'LM': ['Wide Playmaker', 'Wide Supporter'],
    'RM': ['Wide Playmaker', 'Wide Supporter'],
    'LW': ['Defensive Winger', 'Direct Winger', 'Creative Winger'],
    'RW': ['Defensive Winger', 'Direct Winger', 'Creative Winger'],
    'ST': ['Poacher', 'Complete Forward', 'Target Man']
}

player_role_map = {
    player: [role for roles in position_roles.values() for role in roles]
    for player in matrices['minutes'].index
}

class RoleOptimizer:
    def __init__(self, matrices, position_roles, player_role_map):
        self.matrices = matrices
        self.position_roles = position_roles
        self.player_role_map = player_role_map

    def combine_matrices(self, selected_matrices_weights):
        combined = None
        total_weight = sum(selected_matrices_weights.values())
        if total_weight == 0:
            return pd.DataFrame()
        
        for m, w in selected_matrices_weights.items():
            if m not in self.matrices:
                raise ValueError(f"Matrix '{m}' not loaded.")
            if combined is None:
                combined = self.matrices[m] * w
            else:
                combined += self.matrices[m] * w
        return combined / total_weight

    def validate_formation(self, formation):
        total_players = sum(formation.values())
        if total_players != 11:
            raise ValueError(f"Total players in formation must be 11, got {total_players}.")
        for pos in formation:
            if pos not in self.position_roles:
                raise ValueError(f"Position '{pos}' not recognized.")

    def _find_best_role_combination(self, combined_matrix, initial_pos_roles, pos_to_optimize_indices):
        best_roles_combo = None
        best_score = -np.inf

        roles_to_check = [self.position_roles.get(initial_pos_roles[i][0]) for i in pos_to_optimize_indices]

        for roles_combo in product(*roles_to_check):
            current_roles = list(initial_pos_roles)
            for i, role in zip(pos_to_optimize_indices, roles_combo):
                current_roles[i] = (current_roles[i][0], role)
            
            best_players_for_roles = []
            for pos, role in current_roles:
                candidates = [p for p in combined_matrix.index if role in self.player_role_map.get(p, [])]
                if not candidates:
                    candidates = combined_matrix.index
                
                best_player = None
                best_player_score = -np.inf
                for player in candidates:
                    player_idx = combined_matrix.index.get_loc(player)
                    score = combined_matrix.iloc[player_idx].mean()
                    if score > best_player_score:
                        best_player_score = score
                        best_player = player
                best_players_for_roles.append(best_player)
            
            score = self.avg_pairwise_score(best_players_for_roles, combined_matrix)
            
            if score > best_score:
                best_score = score
                best_roles_combo = roles_combo

        final_roles = list(initial_pos_roles)
        for i, role in zip(pos_to_optimize_indices, best_roles_combo):
            final_roles[i] = (final_roles[i][0], role)
            
        return final_roles, best_score

    def avg_pairwise_score(self, lineup, combined_matrix):
        idxs = [combined_matrix.index.get_loc(p) for p in lineup]
        score_sum = 0
        count = 0
        for i in range(len(idxs)):
            for j in range(i + 1, len(idxs)):
                a, b = idxs[i], idxs[j]
                score_sum += (combined_matrix.iloc[a, b] + combined_matrix.iloc[b, a]) / 2
                count += 1
        return score_sum / count if count > 0 else 0

    def optimize_roles(self, selected_matrices_weights, positions_and_roles):
        formation_counts = {}
        for pos, _ in positions_and_roles:
            formation_counts[pos] = formation_counts.get(pos, 0) + 1
        self.validate_formation(formation_counts)

        combined_matrix = self.combine_matrices(selected_matrices_weights)
        
        positions_to_optimize_indices = [i for i, (pos, role) in enumerate(positions_and_roles) if role is None]

        if not positions_to_optimize_indices:
            print("All roles have been specified. No role optimization needed.")
            return positions_and_roles, 0.0

        print(f"Optimizing roles for {len(positions_to_optimize_indices)} positions...")

        final_roles, best_score = self._find_best_role_combination(
            combined_matrix, positions_and_roles, positions_to_optimize_indices
        )
        
        return final_roles, best_score

def new_input_method():
    print("Available matrices:")
    keys = list(matrices.keys())
    for i, m in enumerate(keys, 1):
        print(f" {i}. {m}")
    selected = input("Enter matrix numbers separated by commas (e.g. 1,3): ").strip()
    selected_indices = [int(x) for x in selected.split(",") if x.strip().isdigit()]
    chosen_matrices = []
    for i in selected_indices:
        if 1 <= i <= len(keys):
            chosen_matrices.append(keys[i-1])
    if not chosen_matrices:
        print("No matrices selected, exiting.")
        exit()

    weights = {}
    print("Enter weight for each selected matrix (weights will be normalized):")
    for m in chosen_matrices:
        while True:
            w = input(f" Weight for '{m}': ").strip()
            try:
                w = float(w)
                if w < 0:
                    print(" Weight must be non-negative")
                    continue
                weights[m] = w
                break
            except:
                print(" Please enter a valid number.")
    total = sum(weights.values())
    if total == 0:
        print("All weights zero, defaulting to equal weights.")
        weights = {m:1 for m in chosen_matrices}
        total = sum(weights.values())
    for m in weights:
        weights[m] /= total

    # Get goalkeeper position and role
    while True:
        pos_gk = input("Define position for Goalkeeper (must be 'GK'): ").strip()
        if pos_gk == 'GK':
            break
        print("Position for goalkeeper must be 'GK'. Try again.")

    gk_roles = position_roles.get('GK', [])
    print(f"Available roles for GK: {gk_roles}")
    role_gk = input("Choose role for GK (or press Enter to skip): ").strip()
    if role_gk not in gk_roles:
        role_gk = None

    while True:
        shape = input("Enter formation shape as D,M,A (e.g. 4,3,3): ").strip()
        parts = shape.split(',')
        if len(parts) != 3:
            print("Please enter exactly 3 numbers separated by commas.")
            continue
        try:
            d, m, a = map(int, parts)
            if d < 0 or m < 0 or a < 0 or d + m + a != 10:
                print("Numbers must be non-negative and total 10 (excluding GK).")
                continue
            break
        except:
            print("Please enter valid integers.")

    defenders = ['LB', 'CB', 'RB']
    midfielders = ['CDM', 'CM', 'CAM', 'LM', 'RM']
    attackers = ['LW', 'RW', 'ST']

    def get_positions_and_roles(num, category, allowed_positions):
        print(f"Define {category} positions and optional roles:")
        pos_roles = []
        for i in range(num):
            while True:
                pos = input(f" Position for {category.lower()} player {i+1} (options: {', '.join(allowed_positions)}): ").strip()
                if pos in allowed_positions:
                    roles = position_roles.get(pos, [])
                    print(f"Available roles for {pos}: {roles}")
                    role = input(f"Choose role for {pos} (or press Enter to skip): ").strip()
                    if role not in roles:
                        role = None
                    pos_roles.append((pos, role))
                    break
                else:
                    print("Invalid position. Try again.")
        return pos_roles

    def_pos_roles = get_positions_and_roles(d, 'Defense', defenders)
    mid_pos_roles = get_positions_and_roles(m, 'Midfield', midfielders)
    att_pos_roles = get_positions_and_roles(a, 'Attack', attackers)

    return chosen_matrices, weights, pos_gk, role_gk, d, m, a, def_pos_roles, mid_pos_roles, att_pos_roles

def main():
    optimizer = RoleOptimizer(matrices, position_roles, player_role_map)

    chosen_matrices, weights, pos_gk, role_gk, d, m, a, def_pos_roles, mid_pos_roles, att_pos_roles = new_input_method()

    positions_and_roles = [(pos_gk, role_gk)] + def_pos_roles + mid_pos_roles + att_pos_roles

    optimized_roles, best_score = optimizer.optimize_roles(weights, positions_and_roles)

    print("\nRole Optimization complete.")
    print(f"Best synergy score for the optimized roles: {best_score:.4f}")
    print(f"Optimized Team Roles:")
    for pos, role in optimized_roles:
        print(f" {pos} : {role}")

if __name__ == "__main__":
    main()


Available matrices:
 1. minutes
 2. win
 3. passes
 4. xg_for
 5. xg_against
 6. shot_assist
 7. shot_dribble
 8. carries
 9. defense
 10. recoveries
 11. pressures
Enter matrix numbers separated by commas (e.g. 1,3): 2
Enter weight for each selected matrix (weights will be normalized):
 Weight for 'win': 1
Define position for Goalkeeper (must be 'GK'): GK
Available roles for GK: ['Conservative', 'Shot Stopper', 'Sweeper Keeper']
Choose role for GK (or press Enter to skip): 
Enter formation shape as D,M,A (e.g. 4,3,3): 4, 3, 3
Define Defense positions and optional roles:
 Position for defense player 1 (options: LB, CB, RB): LB
Available roles for LB: ['Balanced Fullback', 'Attacking Fullback', 'Defensive Fullback']
Choose role for LB (or press Enter to skip): Attacking Fullback
 Position for defense player 2 (options: LB, CB, RB): CB
Available roles for CB: ['Stopper', 'Sweeper', 'Enforcer', 'Libero', 'Ball Playing CB']
Choose role for CB (or press Enter to skip): Enforcer
 Position fo